# Сбор аниме данных.

----
![Причина парсинга](..\docs\parsing_reason.jpg)

---

## Оглавление

* ### [Введение](#Введение.)
* ### [Начало работы](#Начало-работы.)
  * #### [Импорт библиотек и создание констант](#Импорт-библиотек-и-создание-констант.)
  * #### [Пример запроса](#Пример-запроса.)
  * #### [Парсинг данных](#Парсинг-данных.)
* ### [Выгрузка датафрейма](#Выгрузка-датафрейма.)

## Введение

В данном ноутбуке произведена работу по парсингу данных с сайта **https://shikimori.one/**

**Поставленная задача:** получить данные о всех аниме-сериалах/полнометражек с сайта.


**Пример сущности для парсинга:**

----
![Пример парсинга](..\docs\anime_entry.jpg)

---

Меня интересует следующая информация:
 * **Название аниме (оба варианта).**
 * **Тип аниме.**
 * **Общее количество эпизодов.**
 * **Количество эпизодов, вышедших в эфир.**
 * **Длительность эпизода/полнометражки.**
 * **Статус аниме.**
 * **Дата начала выпуска.**
 * **Возрастной рейтинг.**
 * **Рейтинг аниме**.
 * **Название студии.**
 * **Оценка людей в разрезе рейтингов.**
 * **В списках у людей.**

**Примечание:** Как можно увидеть, информации для сбора достаточно, благо для парсинга мы будем использовать уже готовую АПИшку от создателя сайта.

## Начало работы.

### Импорт библиотек и создание констант.

In [1]:
import pandas as pd
import numpy as np
import pyprind
import requests
import time

HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
URL = "https://shikimori.one/api/animes/" # url-адресс APIшки для запросов.
EXAMPLE_ANIME_ENTRY_ID = 49596 # id аниме с изображения выше.

### Пример запроса.

В данном пункте, я покажу, что мы должны получать от сервера.

In [2]:
response = requests.get(URL+str(EXAMPLE_ANIME_ENTRY_ID),
                        headers=HEADERS).json()

In [3]:
response # Полный ответ.

{'id': 49596,
 'name': 'Blue Lock',
 'russian': 'Синяя тюрьма: Блю Лок',
 'image': {'original': '/system/animes/original/49596.jpg?1664652076',
  'preview': '/system/animes/preview/49596.jpg?1664652076',
  'x96': '/system/animes/x96/49596.jpg?1664652076',
  'x48': '/system/animes/x48/49596.jpg?1664652076'},
 'url': '/animes/49596-blue-lock',
 'kind': 'tv',
 'score': '8.42',
 'status': 'ongoing',
 'episodes': 24,
 'episodes_aired': 4,
 'aired_on': '2022-10-09',
 'released_on': None,
 'rating': 'pg_13',
 'english': ['Blue Lock'],
 'japanese': ['ブルーロック'],
 'synonyms': [],
 'license_name_ru': None,
 'duration': 24,
 'description': 'Осознав плачевное состояние японского футбола, национальная ассоциация решается на отчаянный шаг и нанимает [character=178715]Дзимпати Эго[/character]. Теперь на плечах этого загадочного и эксцентричного тренера лежит неподъёмная ответственность — привести Японию к победе на чемпионате мира по футболу. Полагая, что залог завоевания выигрыша — это агрессивный нап

In [4]:
response['rates_scores_stats'] # Оценка людей в разрезе рейтингов.

[{'name': 10, 'value': 517},
 {'name': 9, 'value': 229},
 {'name': 8, 'value': 275},
 {'name': 7, 'value': 123},
 {'name': 6, 'value': 40},
 {'name': 5, 'value': 19},
 {'name': 4, 'value': 9},
 {'name': 3, 'value': 8},
 {'name': 2, 'value': 7},
 {'name': 1, 'value': 16}]

In [5]:
response['rates_statuses_stats']  # В списках у людей.

[{'name': 'Запланировано', 'value': 9621},
 {'name': 'Просмотрено', 'value': 136},
 {'name': 'Смотрю', 'value': 7090},
 {'name': 'Брошено', 'value': 116},
 {'name': 'Отложено', 'value': 193}]

In [6]:
response['genres'] # Жанр(ы) аниме.

[{'id': 27, 'name': 'Shounen', 'russian': 'Сёнен', 'kind': 'anime'},
 {'id': 30, 'name': 'Sports', 'russian': 'Спорт', 'kind': 'anime'}]

### Парсинг данных.

In [7]:
n = 48000 # Последний idшник имеет номер на 45 тысячи, взял с запасом.
response_list = [] # Наш массив с ответами.
requset_count = 0 # Кол-во сделанных запросов.
bar = pyprind.ProgBar(n) # Просто отображать прогресс загрузки.

for i in range (1, n):
    requset_count += 1
    # Делаем паузу в 5 секунд на каждом 50-ом запросе для того, чтобы нас не забанили.
    if requset_count % 50 == 0:
        time.sleep(5)
    
    bar.update() # обновляем прогресс-бар.
    
    response = requests.get(URL+str(i),
                        headers=HEADERS)
    
    # В случае если аниме найдено.
    if response.status_code == 200:
        response_list.append(response.json())
    # Пропускаем итерацию.
    else:
        continue

0% [###########################   ] 100% | ETA: 01:02:57

ConnectionError: HTTPSConnectionPool(host='shikimori.one', port=443): Max retries exceeded with url: /api/animes/44498 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000257EE925B80>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

![На случай загруки всех данных](..\docs\2000_years.jpg)

<div class="alert alert-danger" role="alert">
        Мы получили <b>ConnectionError,</b> в дальнейшем стоит убедиться на каком этапе он вышел.
</div>

In [8]:
print(f"Количество полученных данных = {len(response_list)}")

Количество полученных данных = 18151


<div class="alert alert-info" role="alert">
    <b>18 тысяч наблюдений хороший результат</b>, т.к не на всех id существует аниме, <b>за первые 50 запросов он находит лишь 31 наблюдение</b>.
</div>

In [11]:
def create_dataframe(responses):
    """
    Функция для создания датафрейма, исходя из полученных ответов.
    
    responses: список ответов от АПИшки.
    """
    
    # Создание пустых списков для дальнейшего заполнения.
    shikimori_id = []
    kinds = []
    names_eng = []
    names_rus = []
    genres_eng = []
    genres_rus = []
    total_episodes = []
    episodes_aired = []
    duration = []
    status = []
    aired_on = []
    rating = []
    score = []
    studio = []

    count_1r = []
    count_2r = []
    count_3r = []
    count_4r = []
    count_5r = []
    count_6r = []
    count_7r = []
    count_8r = []
    count_9r = []
    count_10r =[]
    
    planned = []
    watched = []
    watching = []
    abandoned = []
    postponed = []

    # Начинаем вытаскивать инфу из каждого ответа.
    for response in responses:
        # Временные массивы с жанрами конкретного аниме.
        tmp_genres_eng = []
        tmp_genres_rus = []
            
        shikimori_id.append(response['id'])
        kinds.append(response['kind'])
        names_eng.append(response['name'])
        names_rus.append(response['russian'])
        
        # Ключ genres хранит в себе другой массив, прийдётся зайти внутрь.
        for genre in response['genres']:
            tmp_genres_eng.append(genre['name'])
            tmp_genres_rus.append(genre['russian'])
            
        genres_eng.append(' '.join(tmp_genres_eng))
        genres_rus.append(' '.join(tmp_genres_rus))
        total_episodes.append(response['episodes'])
        episodes_aired.append(response['episodes_aired'])
        duration.append(response['duration'])
        status.append(response['status'])
        aired_on.append(response['aired_on'])
        rating.append(response['rating'])
        
        # На случай если студия отсутствует.
        try:
            studio.append(response['studios'][0]['name'])
        except:
            studio.append('Studio not found')
        
        """Обернём в try/catch-блоки, т.к не всегда могут быть оценки или голоса"""
        try:
            score.append(response['score'])
        except:
            score.append(0)
        
        try:
            count_1r.append(response['rates_scores_stats'][9]['value']) 
        except:
            count_1r.append(0)
            
        try:
            count_2r.append(response['rates_scores_stats'][8]['value'])
        except:
            count_2r.append(0)  
            
        try:
            count_3r.append(response['rates_scores_stats'][7]['value'])
        except:
            count_3r.append(0)
            
        try:
            count_4r.append(response['rates_scores_stats'][6]['value']) 
        except:
            count_4r.append(0)
            
        try:
            count_5r.append(response['rates_scores_stats'][5]['value']) 
        except:
            count_5r.append(0)
            
        try:
            count_6r.append(response['rates_scores_stats'][4]['value'])
        except:
            count_6r.append(0)
            
        try:
            count_7r.append(response['rates_scores_stats'][3]['value']) 
        except:
            count_7r.append(0) 
            
        try:
            count_8r.append(response['rates_scores_stats'][2]['value'])
        except:
            count_8r.append(0)
        
        try:
            count_9r.append(response['rates_scores_stats'][1]['value'])
        except:
            count_9r.append(0
                           )
        try:
            count_10r.append(response['rates_scores_stats'][0]['value'])
        except:
            count_10r.append(0)
        
        try:
            planned.append(response['rates_statuses_stats'][0]['value'])
        except:
            planned.append(0)
        
        try:
            watched.append(response['rates_statuses_stats'][1]['value'])
        except:
            watched.append(0)
            
        try:
            watching.append(response['rates_statuses_stats'][2]['value'])
        except:
            watching.append(0)
        
        try:
            abandoned.append(response['rates_statuses_stats'][3]['value'])
        except:
            abandoned.append(0)
            
        try:
            postponed.append(response['rates_statuses_stats'][4]['value'])
        except:
            postponed.append(0)
        
    # Формируем датафрейм из заполненных списков.
    return pd.DataFrame({
        'shikimori_id': shikimori_id,
        'studio': studio,
        'name_eng': names_eng,
        'name_rus': names_rus,
        'kind': kinds,
        'genres_eng': genres_eng,
        'genres_rus': genres_rus,
        'rating': rating,
        'status': status,
        'total_episodes': total_episodes,
        'episodes_aired': episodes_aired,
        'duration_min' : duration,
        'aired_on': pd.to_datetime(aired_on),
        'score': np.float64(score),
        'count_1r': count_1r,
        'count_2r': count_2r,
        'count_3r': count_3r,
        'count_4r': count_4r,
        'count_5r': count_5r,
        'count_6r': count_6r,
        'count_7r': count_7r,
        'count_8r': count_8r,
        'count_9r': count_9r,
        'count_10r': count_10r,
        'planned': planned,
        'watched': watched,
        'watching': watching,
        'abandoned': abandoned,
        'postponed': postponed
    })

In [12]:
create_dataframe(response_list)

,shikimori_id,studio,name_eng,name_rus,kind,genres_eng,genres_rus,rating,status,total_episodes,...,count_6r,count_7r,count_8r,count_9r,count_10r,planned,watched,watching,abandoned,postponed
0,1,Sunrise,Cowboy Bebop,Ковбой Бибоп,tv,Action Adventure Comedy Drama Sci-Fi Space,Экшен Приключения Комедия Драма Фантастика Космос,r_plus,released,26,...,1256,3676,8136,11547,19832,37941,67131,10921,3592,4803
1,5,Bones,Cowboy Bebop: Tengoku no Tobira,Ковбой Бибоп: Достучаться до небес,movie,Action Drama Mystery Sci-Fi Space,Экшен Драма Детектив Фантастика Космос,r,released,1,...,554,1749,3864,3880,4208,11998,25857,419,254,385
2,6,Madhouse,Trigun,Триган,tv,Shounen Action Adventure Comedy Drama Sci-Fi,Сёнен Экшен Приключения Комедия Драма Фантастика,pg_13,released,26,...,909,2865,5467,5287,6537,25542,36121,3546,1846,1988
3,7,Sunrise,Witch Hunter Robin,Робин — охотница на ведьм,tv,Action Drama Mystery Supernatural Police,Экшен Драма Детектив Сверхъестественное Полиция,pg_13,released,26,...,348,708,699,316,284,11093,4137,921,662,613
4,8,Toei Animation,Bouken Ou Beet,Приключения короля Бита,tv,Shounen Adventure Fantasy Supernatural,Сёнен Приключения Фэнтези Сверхъестественное,pg,released,52,...,18,38,20,9,26,441,249,55,51,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18146,44492,Studio not found,Yu Qi Lin: Xian Ling Qi Yu,,tv,Kids,Детское,pg,released,52,...,0,0,0,0,0,6,7,0,0,0
18147,44493,Studio not found,Zhandou Ba Ling Shou,,tv,Mecha Kids,Меха Детское,pg,released,52,...,0,0,0,0,0,5,8,0,0,0
18148,44494,Studio not found,Gang Jia Xiaolong Xia,,tv,Sci-Fi Kids,Фантастика Детское,pg,released,52,...,0,0,0,0,0,5,8,0,0,0
18149,44495,Studio not found,Shouwang Zhengfeng,,tv,Sci-Fi Kids,Фантастика Детское,pg,released,26,...,0,0,0,0,0,5,9,0,0,0


Судя по последнему idшнику, мы получили практически все аниме.

Попробую найти инфу про несколько аниме 
 * https://shikimori.one/animes/44511-chainsaw-man 
 * https://shikimori.one/animes/41467-bleach-sennen-kessen-hen
 * https://shikimori.one/animes/z5114-fullmetal-alchemist-brotherhood

In [15]:
test = create_dataframe(response_list)
test[test['shikimori_id'] == 44511]

,shikimori_id,studio,name_eng,name_rus,kind,genres_eng,genres_rus,rating,status,total_episodes,...,count_6r,count_7r,count_8r,count_9r,count_10r,planned,watched,watching,abandoned,postponed


In [16]:
test[test['shikimori_id'] == 41467]

,shikimori_id,studio,name_eng,name_rus,kind,genres_eng,genres_rus,rating,status,total_episodes,...,count_6r,count_7r,count_8r,count_9r,count_10r,planned,watched,watching,abandoned,postponed
16628,41467,Pierrot,Bleach: Sennen Kessen-hen,Блич: Тысячелетняя кровавая война,tv,Shounen Action Adventure Comedy Supernatural S...,Сёнен Экшен Приключения Комедия Сверхъестестве...,r,ongoing,0,...,41,103,184,271,1458,14258,403,8200,119,301


In [17]:
test[test['shikimori_id'] == 5114]

,shikimori_id,studio,name_eng,name_rus,kind,genres_eng,genres_rus,rating,status,total_episodes,...,count_6r,count_7r,count_8r,count_9r,count_10r,planned,watched,watching,abandoned,postponed
3961,5114,Bones,Fullmetal Alchemist: Brotherhood,Стальной алхимик: Братство,tv,Shounen Action Adventure Comedy Drama Fantasy ...,Сёнен Экшен Приключения Комедия Драма Фэнтези ...,r,released,64,...,1476,4279,10073,16665,36862,38272,119325,10852,4797,4672


<div class="alert alert-warning" role="alert">
    В датафрейме отсутствует <b>маленькая часть тайтлов</b>, и не обязательно новинки (чем больше id != новее аниме). Думаю <b>можно пропустить дополнительную загрузку некоторых тайтлов.</b>
</div>

## Выгрузка датафрейма.

<div class="alert alert-success" role="alert">
    Наши данные получены, теперь их можно выгрузить в <b>CSV-файл</b>.
</div>

In [18]:
data = create_dataframe(response_list)
data.to_csv("../data/parsed_data.csv", index=False)